In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
! pip install datasets transformers[sentencepiece] sacrebleu sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 30.6 MB/s 
     |████████████████████████████████| 5.3 MB 55.9 MB/s 
     |████████████████████████████████| 118 kB 71.5 MB/s 
     |████████████████████████████████| 880 kB 60.4 MB/s 
     |████████████████████████████████| 212 kB 71.3 MB/s 
     |████████████████████████████████| 163 kB 70.3 MB/s 
     |████████████████████████████████| 115 kB 72.6 MB/s 
     |████████████████████████████████| 127 kB 70.9 MB/s 
     |████████████████████████████████| 7.6 MB 48.6 MB/s 
     |████████████████████████████████| 1.3 MB 61.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ba41b180b2afcaec0cc214c0b472de9fbbad8e944c9e06b2db3c3ea7372ce221
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: 

In [ ]:
#model_checkpoint = "/content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en"
model_checkpoint = "Helsinki-NLP/opus-mt-ur-en"

In [ ]:
from datasets import load_dataset, load_metric
from sklearn.metrics import precision_recall_fscore_support

#raw_datasets = load_dataset("opus100", "en-ur")
raw_datasets = load_dataset('json', data_files={
    #'train':['/content/drive/MyDrive/Colab Notebooks/Tanzil/Tanzil_formatted.json'],
    #'validation':['/content/drive/MyDrive/Colab Notebooks/Tanzil/Tanzil_val_formatted.json'],
    'test': ['/content/drive/MyDrive/Colab Notebooks/tico19-testset/tico19-testset/test/test_formatted.json']})


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-7fb17b15874d421a/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2100
    })
})

In [ ]:
raw_datasets['test'][0]

{'translation': {'en': 'about how long have these symptoms been going on?\r',
  'ur': 'یہ علامات کتنے عرصے سے جاری ہیں؟\r'}}

In [ ]:
metric1 = load_metric("sacrebleu") # high scores are better
metric2 = load_metric("ter")  # high score are worse
metric3 = load_metric("chrf")
metric4 = load_metric("meteor")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["test"])

,translation
0,"{'en': '""However, not all systems with central servers need to have access to personal location data; a number of privacy-preserving systems have been created that use central servers only for intercommunication (see section below)."" ', 'ur': 'بہرحال، مرکزی سرورز کے ساتهـ مربوط تمام سسٹم کو ذاتی موقع محل کی ڈیٹا تک رسائی کی ضرورت نہیں پڑتی؛ رازداری کے تحفظ کے متعدد نظام ایسے بهی بنائے گئے ہیں جو آپس میں خبررسانی کے لیے صرف مرکزی سرورز استعمال کرتے ہیں (نیچے والا حصہ دیکهیں)۔ '}"
1,"{'en': '""Finally, since many mammals including domestic animals might be susceptible to SARS-CoV-2, both surveillance and experimental infection should be conducted."" ', 'ur': 'آخر میں، کیونکہ گھریلو جانوروں سمیت بہت سارے دوسرے دودھ پلانے والے جانور SARS-CoV-2 کے لیے حساس ہو سکتے ہیں، اس لیے انفیکشن کے لیے ان کی نگرانی اور تجربات دونوں منعقد کیے جانے چاہیے۔ '}"
2,"{'en': 'Frequent use of alcohol-based hand sanitizers can cause dry skin unless emollients and/or skin moisturizers are added to the formula. ', 'ur': 'الکحل پر مبنی سینی ٹایزر کے زیادہ استعمال سے جلد خشک ہو سکتی ہے،جب تک کہ فارمولے میں کریم اور/یا جلد گیلا کرنے والی اشیا شامل نہ کی جایں۔ '}"
3,"{'en': '""In contrast to the role of camels in the transmission of MERS-CoV, the role of pangolins, if there is any, in the transmission of SARS-CoV-2 is different."" ', 'ur': 'MERS-CoV کی منتقلی میں اونٹوں کے کردار کے برعکس، SARS-CoV-2 کی منتقلی میں، پینگولن کا کردار، اگر کوئی ہے تو، مختلف ہے۔ '}"
4,"{'en': '""On 14 February, a WHO-led Joint Mission Team with China was activated to provide international and WHO experts on the ground in China to assist in the domestic management and evaluate """"the severity and the transmissibility of the disease"""" by hosting workshops and meetings with key national-level institutions and to conduct field visits to assess the """"impact of response activities at provincial and county levels, including urban and rural settings"""".On 25 February, the WHO declared that """"the world should do more to prepare for a possible coronavirus pandemic,"""" stating that while it was still too early to call it a pandemic, countries should nonetheless be """"in a phase of preparedness""""."" ', 'ur': '""14 فروری کو، WHO کی زیر قیادت ایک مشترکہ مشن ٹیم کو چین کے ساتھ فعال کیا گیا تاکہ بین الاقوامی اور WHO کے ماہرین چینکی سرزمین پر گھریلو تنظیم میں مدد فراہم کرسکیں اور قومی سطح کے اہم اداروں کے ساتھ ورکشاپوں اور ملاقاتوں کی میزبانی کے ذریعے """"بیماری کی شدت اور منتقلی"""" کا جائزہ لے سکیں اور """"صوبائی اور ضلعی سطح پر جوابی سرگرمیوں کے اثرات کا اندازہ لگانے کے لئے ان علاقوں کا دوره کریں، بشمول شہری اور دیہاتی جگہوں کے""""- 25 فروری کو، عالمی اداره صحت نے اعلان کیا کہ """"دنیا کو ممکنہ کوروناوائرس عالمی وبا کی تیاری کے لئے مزید کام کرنا چاہئے،"""" یہ کہتے ہوئے کہ اگرچہ یہ ابهی وقت سے پہلے ہوگا کہ اس بیماری کو وبائی مرض کہا جائے، تاہم ممالک کو بہرحال """"تیاری کے مرحلے میں"""" ہونا چاہئے۔"" '}"


In [ ]:
from transformers import MarianTokenizer, MarianModel
    
tokenizer = MarianTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/848k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/816k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='Helsinki-NLP/opus-mt-ur-en', vocab_size=62025, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'})

In [ ]:
from transformers import MarianMTModel, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

tokens = tokenizer.encode('This is a input.', return_tensors='pt')
print("These are tokens!", tokens)
for token in tokens[0]:
    print("This are decoded tokens!", tokenizer.decode([token]))

model = MarianMTModel.from_pretrained(model_checkpoint)
print(model.model.config)
#for e in model.embeddings.word_embeddings(tokens)[0]:
#    print("This is an embedding!", e)

These are tokens! tensor([[ 101, 2023, 2003, 1037, 7953, 1012,  102]])
This are decoded tokens! [CLS]
This are decoded tokens! this
This are decoded tokens! is
This are decoded tokens! a
This are decoded tokens! input
This are decoded tokens! .
This are decoded tokens! [SEP]
MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-ur-en",
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      62024
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 62024,
  "decoder_vocab_size": 62025,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_i

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertModel

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
tokens = tokenizer.encode('This is a input.', return_tensors='pt')
print("These are tokens!", tokens)
for token in tokens[0]:
    print("This are decoded tokens!", tokenizer.decode([token]))

model = DistilBertModel.from_pretrained("distilbert-base-uncased")
print(model)
#for e in model.embeddings.word_embeddings(tokens)[0]:
#    print("This is an embedding!", e)

These are tokens! tensor([[ 101, 2023, 2003, 1037, 7953, 1012,  102]])
This are decoded tokens! [CLS]
This are decoded tokens! this
This are decoded tokens! is
This are decoded tokens! a
This are decoded tokens! input
This are decoded tokens! .
This are decoded tokens! [SEP]


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [ ]:
urdu_example = '''اے حبیب مکرم ! ہم آپ کو خود ایسا پڑھائیں گے کہ آپ کبھی نہیں بھولیں گے ۔'''
urdu_tokens = tokenizer.tokenize(urdu_example)
urdu_tokens

['▁اے',
 '▁حبیب',
 '▁مکرم',
 '▁!',
 '▁ہم',
 '▁آپ',
 '▁کو',
 '▁خود',
 '▁ایسا',
 '▁پڑھائیں',
 '▁گے',
 '▁کہ',
 '▁آپ',
 '▁کبھی',
 '▁نہیں',
 '▁بھولیں',
 '▁گے',
 '▁۔']

In [ ]:
eng_example ="(O Esteemed Beloved!) We shall teach you (in a way) that nothing shall you (ever) forget."
eng_tokens = tokenizer.tokenize(eng_example)
eng_tokens

['▁(',
 'O',
 '▁E',
 'st',
 'eem',
 'ed',
 '▁Be',
 'love',
 'd',
 '!)',
 '▁We',
 '▁sh',
 'all',
 '▁',
 'te',
 'a',
 'ch',
 '▁you',
 '▁(',
 'in',
 '▁a',
 '▁',
 'way',
 ')',
 '▁that',
 '▁no',
 'th',
 'ing',
 '▁sh',
 'all',
 '▁you',
 '▁(',
 'e',
 'ver',
 ')',
 '▁for',
 'get',
 '.']

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "ur"
target_lang = "en"
prefix = ""

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
#preprocess_function(raw_datasets['train'][:2])
preprocess_function(raw_datasets['test'][:2])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


{'input_ids': [[45, 10035, 2810, 4481, 106, 11, 1483, 21, 189, 0], [4, 6697, 12, 428, 21724, 29, 318, 79, 4996, 1076, 1383, 1019, 395, 30, 88, 12, 1460, 13, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[271, 357, 1075, 55, 460, 14256, 203, 1266, 95, 75, 0], [7, 120, 10228, 4552, 283, 51, 6287, 140, 321, 3589, 65, 76, 1742, 0]]}

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
#small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(30000))
#small_valid_dataset = tokenized_datasets["validation"]#[0:1500] #.shuffle(seed=42).select(range(2000))
#small_test_dataset = tokenized_datasets["test"]

In [ ]:
from transformers import MarianMTModel, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = MarianMTModel.from_pretrained(model_checkpoint)


Downloading:   0%|          | 0.00/306M [00:00<?, ?B/s]

In [ ]:
model.config.activation_dropout = 0.01
model.config.add_final_layer_norm = True
model.config.attention_dropout = 0.01
model.config.add_cross_attention = True

In [ ]:
#model.config.early_stopping 

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"finetuned-marian-mt-{source_lang}-{target_lang}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch", 
    learning_rate=2e-5,
    optim = "adamw_torch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.001,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    generation_num_beams=6,
    load_best_model_at_end= True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    s_bleu_result = metric1.compute(predictions=decoded_preds, references=decoded_labels)
    #s_bleu_result = {k: round(v, 2) for k, v in s_bleu_result.items()}

    ter_result = metric2.compute(predictions=decoded_preds, references=decoded_labels, ignore_punct=True, case_sensitive=False)
    #ter_result = {k: round(v, 2) for k, v in ter_result.items()}

    chrf = metric3.compute(predictions=decoded_preds, references=decoded_labels)
    #chrf = {k: round(v, 2) for k, v in chrf.items()}
    
    meteor = metric4.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_result = round(meteor['meteor'], 2)

    #google_bleu_score = metric5.compute(predictions=decoded_preds, references=decoded_labels)
    #google_bleu_result = round(google_bleu_score['google_bleu'], 2)
    #preds = pred.predictions.argmax(-1)
    #a_preds = np.argmax(preds, axis=-1)
    #precision, recall, f1, _ = precision_recall_fscore_support(labels, a_preds)
    #accuracy = accuracy_score(labels, a_preds)

    
    #accuracy = metric4.compute(predictions=a_preds, references=labels)
    #precision = metric5.compute(predictions=a_preds, references=labels)
    #recall = metric6.compute(predictions=a_preds, references=labels)

    result = {"bleu": round(s_bleu_result["score"],2), "chrF": round(chrf["score"],2) ,"TER": round(ter_result["score"],2), 
              "METEOR": meteor_result}#,"Google Bleu": google_bleu_result["google_bleu"]}#,"recall": bert_score["recall"],"f1": bert_score["f1"]}
    #prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    #result["gen_len"] = np.mean(prediction_lens)
    
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=small_train_dataset,
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 30000
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 37500


Epoch,Training Loss,Validation Loss,Bleu,Chrf,Ter,Meteor
1,0.238700,0.183916,14.680000,41.450000,82.470000,0.370000
2,0.205200,0.184925,16.360000,42.180000,79.620000,0.370000
3,0.186700,0.195640,15.990000,41.860000,79.640000,0.370000
4,0.169000,0.202889,15.920000,41.810000,78.630000,0.370000
5,0.156100,0.208910,16.610000,41.990000,77.210000,0.370000
6,0.147600,0.219800,15.170000,41.130000,79.090000,0.360000
7,0.142600,0.223101,16.060000,41.670000,78.160000,0.370000
8,0.129700,0.232900,15.060000,40.950000,79.110000,0.360000
9,0.121500,0.234172,15.690000,41.320000,78.280000,0.360000
10,0.114100,0.235708,15.600000,41.190000,78.030000,0.360000


The following columns in the evaluation set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 16
Saving model checkpoint to finetuned-marian-mt-ur-en/checkpoint-1875
Configuration saved in finetuned-marian-mt-ur-en/checkpoint-1875/config.json
Model weights saved in finetuned-marian-mt-ur-en/checkpoint-1875/pytorch_model.bin
tokenizer config file saved in finetuned-marian-mt-ur-en/checkpoint-1875/tokenizer_config.json
Special tokens file saved in finetuned-marian-mt-ur-en/checkpoint-1875/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****

TrainOutput(global_step=37500, training_loss=0.1252547240193685, metrics={'train_runtime': 12976.2543, 'train_samples_per_second': 46.238, 'train_steps_per_second': 2.89, 'total_flos': 1.4862933018279936e+16, 'train_loss': 0.1252547240193685, 'epoch': 20.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-tanzil-trained")

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-tanzil-trained
Configuration saved in /content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-tanzil-trained/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-tanzil-trained/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-tanzil-trained/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-tanzil-trained/special_tokens_map.json


In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 16


{'epoch': 20.0,
 'eval_METEOR': 0.37,
 'eval_TER': 82.47,
 'eval_bleu': 14.68,
 'eval_chrF': 41.45,
 'eval_loss': 0.18391641974449158,
 'eval_runtime': 188.4539,
 'eval_samples_per_second': 7.96,
 'eval_steps_per_second': 0.499}

In [ ]:
model_checkpoint = "/content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-pib-trained"
#model_checkpoint ="Helsinki-NLP/opus-mt-ur-en"
from transformers import MarianTokenizer, MarianMTModel
    
tokenizer = MarianTokenizer.from_pretrained(model_checkpoint)
model = MarianMTModel.from_pretrained(model_checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
model.config

MarianConfig {
  "_name_or_path": "/content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-pib-trained",
  "activation_dropout": 0.01,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_cross_attention": true,
  "add_final_layer_norm": true,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.01,
  "bad_words_ids": [
    [
      62024
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 62024,
  "decoder_vocab_size": 62025,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "extra_pos_embeddings": 62025,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": tr

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2100
  Batch size = 16


{'eval_loss': 5.186152458190918,
 'eval_bleu': 8.13,
 'eval_chrF': 32.43,
 'eval_TER': 89.31,
 'eval_METEOR': 0.28,
 'eval_runtime': 558.2769,
 'eval_samples_per_second': 3.762,
 'eval_steps_per_second': 0.236}

In [ ]:
trainer.compute_loss_context_manager

<bound method Trainer.compute_loss_context_manager of <transformers.trainer_seq2seq.Seq2SeqTrainer object at 0x7f6f93dc6a90>>

In [ ]:
from transformers import pipeline

translator = pipeline("translation_ur_to_en", model=model_checkpoint)

input = "آندھراپردیش میں ہوئے ایک حادثے میں مہلوکین کیلئے وزیراعظم کی تعزیت "
ground = "PM condoles loss of lives in an accident in Andhra Pradesh"
output = translator(input)
print("Input:  " + input + "\n")
print("Output:  " + output[0]['translation_text'] + "\n")
print("Ground:  " + ground + "\n")

Input:  آندھراپردیش میں ہوئے ایک حادثے میں مہلوکین کیلئے وزیراعظم کی تعزیت 

Output:  PM condoles Mahotssamen in an accident accident in Andhra Pradesh

Ground:  PM condoles loss of lives in an accident in Andhra Pradesh



In [ ]:
from transformers import pipeline
model_checkpoint = "Helsinki-NLP/opus-mt-ur-en"
translator = pipeline("translation_ur_to_en", model=model_checkpoint)

input = "آندھراپردیش میں ہوئے ایک حادثے میں مہلوکین کیلئے وزیراعظم کی تعزیت "
ground = "PM condoles loss of lives in an accident in Andhra Pradesh"
output = translator(input)
print("Input:  " + input + "\n")
print("Output:  " + output[0]['translation_text'] + "\n")
print("Ground:  " + ground + "\n")

Input:  آندھراپردیش میں ہوئے ایک حادثے میں مہلوکین کیلئے وزیراعظم کی تعزیت 

Output:  In an accident in São Paulo, at the time of a accident, the prime minister for the prime minister in São Paulo.

Ground:  PM condoles loss of lives in an accident in Andhra Pradesh

